In [1]:
!pip install transformers

In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch import nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import time
import datetime

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
data = pd.read_csv("/kaggle/input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv")

In [4]:
data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [5]:
data.shape

(20491, 2)

In [6]:
import torch

# 클래스 레이블
labels = data['Rating']

# 클래스의 개수
num_classes = max(labels) + 1

# 원-핫 인코딩
onehot_labels = torch.eye(num_classes)[data['Rating']]
onehot_labels = onehot_labels[:, 1:]
print(onehot_labels)

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['Review'], onehot_labels, test_size=0.2, stratify = onehot_labels, random_state=42)

In [8]:
# BERT의 입력 형식
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in x_train]

In [9]:
# BERT 토크나이저로 문장을 토큰화
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in tqdm(sentences)]

print (sentences[0])
print (tokenized_texts[0])

100%|██████████| 16392/16392 [01:00<00:00, 270.27it/s]

[CLS] san francisco charm stay san francisco frequently business, tried joie vivre hotels love old world charm elegance, rex great, small staff knows conviences needed i.e. restaurant lobby bar internet large clean rooms welcoming staff, right corner union square, stay want charm old san francisco,   [SEP]
['[CLS]', 'san', 'francisco', 'charm', 'stay', 'san', 'francisco', 'frequently', 'business', ',', 'tried', 'jo', '##ie', 'vi', '##vre', 'hotels', 'love', 'old', 'world', 'charm', 'elegance', ',', 'rex', 'great', ',', 'small', 'staff', 'knows', 'con', '##vie', '##nce', '##s', 'needed', 'i', '.', 'e', '.', 'restaurant', 'lobby', 'bar', 'internet', 'large', 'clean', 'rooms', 'welcoming', 'staff', ',', 'right', 'corner', 'union', 'square', ',', 'stay', 'want', 'charm', 'old', 'san', 'francisco', ',', '[SEP]']


In [10]:
# 입력 최대 토큰 개수
MAX_LEN = 128

# 토큰을 id로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(tokenized_texts)]

# 패딩
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

100%|██████████| 16392/16392 [00:03<00:00, 5421.33it/s]


array([  101,  2624,  3799, 11084,  2994,  2624,  3799,  4703,  2449,
        1010,  2699,  8183,  2666,  6819, 12229,  9275,  2293,  2214,
        2088, 11084, 27745,  1010, 10151,  2307,  1010,  2235,  3095,
        4282,  9530, 13469,  5897,  2015,  2734,  1045,  1012,  1041,
        1012,  4825,  9568,  3347,  4274,  2312,  4550,  4734, 18066,
        3095,  1010,  2157,  3420,  2586,  2675,  1010,  2994,  2215,
       11084,  2214,  2624,  3799,  1010,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [11]:
attention_masks = []

# attention mask if padding = 0; 1
for seq in tqdm(input_ids):
    seq_mask = [int(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

100%|██████████| 16392/16392 [00:01<00:00, 11785.57it/s]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [12]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    np.array(y_train), 
                                                                                    random_state=2018,
                                                                                    stratify = y_train,
                                                                                    test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  2307,  2994, 10931,  3309,  4370, 10931,  1017,  6385, 19227,
         1010, 17414,  3495,  3309,  3446, 14210,  2692, 22563,  2094,  2305,
        15203,  7440,  3193,  2282,  1010,  4638,  1011,  1999,  3733,  2288,
         9725,  7621,  2282, 19227,  1010,  2489,  5835, 12327, 10931,  6945,
        18389,  5909,  7967,  2015,  2994,  2428,  2569,  1010,  3193,  7440,
         2204,  2275, 16317, 27208,  3121,  2215,  2488,  3193,  2994,  3309,
        16317,  1010,  9690,  2106,  1050,  1005,  1056,  2644,  9107,  3193,
         4597, 14406,  3121,  2422,  1010,  4734, 12176,  2154,  2326,  7884,
         2204,  1010,  8823,  3585,  7053,  2822,  3095, 14044, 16755,  2075,
        10447,  2648,  2379,  4770,  2181,  5337,  1010,  2106,  1050,  1005,
         1056,  2338,  7562,  3309,  2074,  2253,  7564, 25964,  2648,  3309,
        16286, 21125,  1010, 12246,  5632,  2994, 16755,  3309,  1010,  4102,
         1019,  1008,  9275,  2204,  3976,  2828,  2282, 17414])

In [13]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [14]:
# BERT 입력 형식
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in x_test]

labels = np.array(y_test)

# 토큰화
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

batch_size = 32

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [15]:
# GPU 디바이스 이름
device_name = tf.test.gpu_device_name()

# GPU 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [16]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [17]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5, problem_type="multi_label_classification")
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [19]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8 # 0으로 나누는 것을 방지
                 )

epochs = 4

# 총 훈련 스텝 = 배치 수 * 에폭
total_steps = len(train_dataloader) * epochs

#스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = np.argmax(labels, axis=1).flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [21]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [22]:
# fix randomseed
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

for epoch_i in range(0, epochs):
    
    #Training
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0

    model.train()
        
    for step, batch in enumerate(tqdm(train_dataloader)):

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    #Validation
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs.logits

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


100%|██████████| 461/461 [02:47<00:00,  2.75it/s]



  Average training loss: 0.36
  Training epcoh took: 0:02:48

Running Validation...
  Accuracy: 0.63
  Validation took: 0:00:06

======== Epoch 2 / 4 ========
Training...


100%|██████████| 461/461 [02:46<00:00,  2.77it/s]



  Average training loss: 0.28
  Training epcoh took: 0:02:47

Running Validation...
  Accuracy: 0.67
  Validation took: 0:00:06

======== Epoch 3 / 4 ========
Training...


100%|██████████| 461/461 [02:46<00:00,  2.76it/s]



  Average training loss: 0.25
  Training epcoh took: 0:02:47

Running Validation...
  Accuracy: 0.66
  Validation took: 0:00:06

======== Epoch 4 / 4 ========
Training...


100%|██████████| 461/461 [02:46<00:00,  2.76it/s]



  Average training loss: 0.22
  Training epcoh took: 0:02:47

Running Validation...
  Accuracy: 0.66
  Validation took: 0:00:06

Training complete!


In [23]:
t0 = time.time()

model.eval()

eval_loss = 0
eval_accuracy = 0
nb_eval_steps = 0
nb_eval_examples = 0

for step, batch in enumerate(tqdm(test_dataloader)):
    
    batch = tuple(t.to(device) for t in batch)
    
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = outputs.logits

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

100%|██████████| 129/129 [00:14<00:00,  8.89it/s]


Accuracy: 0.67
Test took: 0:00:15


In [24]:
def convert_input_data(sentences):

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    MAX_LEN = 128

    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [25]:
def test_sentences(sentences):

    model.eval()

    inputs, masks = convert_input_data(sentences)

    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()

    return logits

In [30]:
#4점 리뷰
review = input()
logits = test_sentences([review])
rating = np.argmax(logits)+1
print("===================================")
print(logits)
print()
print(f"본 리뷰는 {rating} 점으로 예측됩니다.")
print("===================================")

 "Nice hotel, no major complaints: (1) some of the furnishings in the room could use some updating (worn areas on the sofa, etc). and (2) Breakfast buffet had a lot of options, but surprisingly few gluten-free ones (would be great to have things like waffles, baked goods like muffins). Staff are very attentive and they helped us find some good off-the-beaten path restaurants."


[[-5.1121035  -3.82513    -1.6346102   0.37617612 -1.6418087 ]]

본 리뷰는 4 점으로 예측됩니다.


In [31]:
#1점 리뷰
review = input()
logits = test_sentences([review])
rating = np.argmax(logits)+1
print("===================================")
print(logits)
print()
print(f"본 리뷰는 {rating} 점으로 예측됩니다.")
print("===================================")

 Klook is the worst transportation website I have ever booked from airport to hotel. We had to leave the airport to go out in the wet cold night to the pick up Point. I nearly fell from the slippery floor trying to look for the car and pick up point at 8 pm. The driver did not drop us off at the main hotel lobby. He dropped us off at an unknown place saying the front entrance is behind the boom gate at the end of the street. It turned out to be a shopping mall at the end of the street. We asked other pedestrians and they told us to walk back out of the street into another street and walk uphill. For the first time I had to walk uphill with my family with 5 luggages to reach the closest entrance which was the back of the hotel. We complained to the company and they did not do anything. They promised to look into the matter and I found I was crossed out of their communication system. This is definite not a reliable company. I normally don’t write many reviews but when I write I write wit

[[ 1.1867468 -1.3499482 -3.5678453 -3.8017917 -3.7586148]]

본 리뷰는 1 점으로 예측됩니다.
